In [100]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize

In [161]:
file = "Cleaned_Text_Dataset.csv"
cleaned_reviews_df = pd.read_csv(file)
del cleaned_reviews_df["Unnamed: 0"]
cleaned_reviews_df.head()

,review_id,business_id,text,target,words
0,fxWnU4OqONBNoQhEcyazSg,krTHKI0YOpASr4gz2CVWFw,"This location used to be good, several years a...",0.0,this location use to be good several year ago ...
1,FhtER9SGsEYkEhRcs09rsQ,krTHKI0YOpASr4gz2CVWFw,I love Cosi but this Cosi is going down hill f...,0.0,love cosi but this cosi be go down hill fast a...
2,0KlwfaHZyvao41_3S47dyg,w9hS5x1F52Id-G1KTrAOZg,Was not a fan of their cheesesteak. Their wiz ...,0.0,be not fan of their cheesesteak their wiz sauc...
3,2qeje7dttkvREbccHev6Pg,7lwe7n-Yc-V9E_HfLAeylg,"It pains me to write this, but I fear I must.....",0.0,it pain to write this but fear must use to rea...
4,1OR23O0giNcxNbFAi4jgcg,DsKzHnkLKnxZTVsFpts4oA,Cocktails were nice however the bartender Paul...,0.0,cocktail be nice however the bartender paul be...


In [162]:
pos_reviews_df = cleaned_reviews_df[cleaned_reviews_df['target'] == 1]
neg_reviews_df = cleaned_reviews_df[cleaned_reviews_df['target'] == 0]
pos_reviews_df = pos_reviews_df.reset_index(drop=True)
neg_reviews_df = neg_reviews_df.reset_index(drop=True)

In [163]:
pos_reviews = cleaned_reviews_df.words[cleaned_reviews_df.target == 1.0]
neg_reviews = cleaned_reviews_df.words[cleaned_reviews_df.target == 0.0]

In [164]:
pos_reviews

4565    go here with friend on the positive side very ...
4566    love everything about kanella from the delicio...
4567    daaaaaang that be great philly cheesesteak fry...
4568    can never eat whole burrito could not stop eat...
4569    price ncame here for restaurant week and think...
                              ...                        
9125    definitely huge fan of the pok spot ve order f...
9126    this place be super cool so many choice in one...
9127    hookah the hookah be good too bit to prepare b...
9128    the food be absolutely amazing the service be ...
9129    the resurgence of old city start with the litt...
Name: words, Length: 4565, dtype: object

## Stopwords

In [165]:
# Create a list of stop words with stopwords library 
# and adding extra stopwords that is not potentially useful 
my_stop_words = set(stopwords.words('english') + 
                    list(ENGLISH_STOP_WORDS) + 
                    ['super', 'duper', 've', 'like', 'got', 
                     'Cleveland', 'just', 'don', 'really', 
                     'said', 'told', 'ok','came', 'went', 
                     'did', 'didn', 'good'])

## TF-IDF

In [166]:
# Create a vectorizer object to generate term document counts
tfidf_pos = TfidfVectorizer(stop_words=my_stop_words, min_df=10, max_df=0.5, 
                        ngram_range=(1,2), token_pattern='[a-z][a-z]+')

tfidf_neg = TfidfVectorizer(stop_words=my_stop_words, min_df=10, max_df=0.5, 
                        ngram_range=(1,2), token_pattern='[a-z][a-z]+')

In [167]:
# Get the vectors
pos_vectors = tfidf_pos.fit_transform(pos_reviews)
neg_vectors = tfidf_neg.fit_transform(neg_reviews)

C:\Users\AA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['cleveland'] not in stop_words.
  warnings.warn(


In [168]:
pos_df = pd.DataFrame(pos_vectors.todense(), columns=[tfidf_pos.get_feature_names_out()])
neg_df = pd.DataFrame(neg_vectors.todense(), columns=[tfidf_neg.get_feature_names_out()])

In [169]:
# get mean for each column (word): highest means are most important words
col_means_pos = {}
for col in pos_df:
    col_means_pos[col] = pos_df[col].mean()

col_means_neg = {}
for col in neg_df:
    col_means_neg[col] = neg_df[col].mean()

In [170]:
no_top_words = 8

print('Top %d words in POSITIVE reviews:' %no_top_words, end='')
print(sorted(col_means_pos, key=col_means_pos.get, reverse=True)[:no_top_words])

print('Top %d words in NEGATIVE reviews:' %no_top_words, end='')
print(sorted(col_means_neg, key=col_means_neg.get, reverse=True)[:no_top_words])

Top 8 words in POSITIVE reviews:[('place',), ('food',), ('great',), ('love',), ('come',), ('time',), ('order',), ('service',)]
Top 8 words in NEGATIVE reviews:[('order',), ('place',), ('time',), ('come',), ('service',), ('say',), ('bad',), ('make',)]



- Looks like the top words for both positive and negative Yelp reviews have mention topic related to service , place , time and order.

## Apply NMF to find Generate Topics

In [171]:
# change num_topics
num_topics = 6

nmf_pos = NMF(n_components=num_topics)
W_pos = nmf_pos.fit_transform(pos_vectors)
H_pos = nmf_pos.components_

nmf_neg = NMF(n_components=num_topics)
W_neg = nmf_neg.fit_transform(neg_vectors)
H_neg = nmf_neg.components_

In [172]:
def display_topics(model, feature_names, num_topics, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        if topic_idx < num_topics:
            print("{:11}".format("Topic %d:" %(topic_idx)), end='')
            print(", ".join(['{:04.3f}*'.format(topic[i])+feature_names[i] \
                             for i in topic.argsort()[:-no_top_words-1:-1]]))

In [173]:
num_topics = num_topics
num_top_words = 6

print('Top topics + words for Positive reviews')
print('-'*39)
display_topics(nmf_pos, tfidf_pos.get_feature_names_out(), num_topics, num_top_words)

print('\nTop topics + words for Negative reviews')
print('-'*39)
display_topics(nmf_neg, tfidf_neg.get_feature_names_out(), num_topics, num_top_words)

Top topics + words for Positive reviews
---------------------------------------
Topic 0:   0.793*order, 0.636*come, 0.504*delicious, 0.483*chicken, 0.478*try, 0.469*dish
Topic 1:   0.933*sandwich, 0.877*cheesesteak, 0.768*cheese, 0.754*pork, 0.696*steak, 0.634*roast
Topic 2:   2.293*pizza, 0.264*crust, 0.228*slice, 0.197*pizza place, 0.170*pizza pizza, 0.161*pie
Topic 3:   1.239*place, 0.863*market, 0.721*love, 0.515*terminal, 0.393*love place, 0.393*food
Topic 4:   1.553*great, 1.177*food, 0.871*service, 0.532*recommend, 0.491*amazing, 0.463*staff
Topic 5:   0.748*hour, 0.741*happy hour, 0.719*happy, 0.687*beer, 0.572*bar, 0.512*drink

Top topics + words for Negative reviews
---------------------------------------
Topic 0:   1.162*table, 0.773*wait, 0.730*come, 0.693*minute, 0.625*ask, 0.601*reservation
Topic 1:   0.839*chicken, 0.596*taste, 0.536*salad, 0.436*sauce, 0.393*dish, 0.371*rice
Topic 2:   2.148*pizza, 0.300*slice, 0.207*pizza place, 0.205*order pizza, 0.194*crust, 0.142*ch

Topics in Positive reviews Similarity:
- Topics(0) : Order , Delicious , Dish  
- Topics(1) : Sandwich , Cheese , Steak , Pork
- Topics(2) : Pizza , Food
- Topics(3) : Place , Market
- Topics(4) : Service , Staff
- Topics(5) ：Happy hour , Beer

Topics in Negative reviews Similarity:
- Topics(0) : Table , Wait
- Topics(1) : Dish , Salad , Taste
- Topics(2) : Pizza , Food
- Topics(3) : Bar , Drink
- Topics(4) : Order , Time , Delivery
- Topics(5) ：Cheesesteak, Sandwich

With these topis, we can investigate any review in the datasets that, the similarity of topics to the reviews

---

## Testing positive review similar topics

In [174]:
sample_pos_review = pos_reviews_df['text'][99] #randomly select a positive review
print('Sample Positive Review :')
print('-'*25)
print(sample_pos_review)
print('-'*25)
topic_dict = {}
for ind, w in enumerate(W_pos[99]):
    topic_dict[ind] = w
for k in sorted(topic_dict, key=topic_dict.get, reverse=True):
     print('Topic {}: {:.4f}'.format(k, topic_dict[k]))

Sample Positive Review :
-------------------------
What an excellent addition to the Fairmount/Francisville neighborhood collection of bars/restaurants! The fries are great, the salmon entree is seriously one of the best salmon dishes I've ever had, and the boards are a delicious and fun way to sample the menu. They definitely have to work out some kinks in the service, as we were told that a wait for our party of three would be a half hour but it ended up being an hour. And once we were seated we barely saw our waiter who apologized for the slow pace of service multiple times. But it was a Saturday night and their 5th night of business so they definitely get a pass. Oh and my beer-lover husband had great things to say about the in-house brews.
-------------------------
Topic 5: 0.0970
Topic 0: 0.0354
Topic 4: 0.0348
Topic 1: 0.0000
Topic 2: 0.0000
Topic 3: 0.0000


## Testing negative review similar topics

In [175]:
sample_neg_review = neg_reviews_df['text'][99] #randomly select a positive review
print('Sample Negative Review :')
print('-'*25)
print(sample_neg_review)
print('-'*25)
topic_dict = {}
for ind, w in enumerate(W_neg[99]):
    topic_dict[ind] = w
for k in sorted(topic_dict, key=topic_dict.get, reverse=True):
     print('Topic {}: {:.4f}'.format(k, topic_dict[k]))

Sample Negative Review :
-------------------------
This place has a very good chicken cheesesteak wrap, but if you are delivering be cautious of the roof delivery guy. I've had two encounters with the same guy and he is rude. The first time he was impatient, when I work at a hospital and was trying to meet him in the lobby. The second time he shoved the bag in my had since I told him to come inside instead of me going outside for the food . That is not how you treat customers! I called and spoke with the manager and he said "well he is very busy, that's no excuse I'll talk to him". If that was no excuse then it shouldn't have been mentioned. I'd recommend only dining in , if you must.
-------------------------
Topic 4: 0.0672
Topic 5: 0.0288
Topic 3: 0.0267
Topic 0: 0.0080
Topic 2: 0.0038
Topic 1: 0.0000


---

## Apply NMF to find General Topic
### with words replace with defined dictionary 'dicty'

In [176]:
dicty = {'pizzas':'pizza', 'burgers':'burger', 'ordered':'order',
         'prices':'price', 'topping':'toppings'}
def replace_words(text, dicty):
    '''Returns text with word substitutions made according
       to the passed `dicty` dictionary'''
    for i,j in dicty.items():
        text = text.replace(i,j)
    return text

# replace words with defined dictionary 'dicty'
pos_reviews = [replace_words(w, dicty) for w in pos_reviews]
neg_reviews = [replace_words(w, dicty) for w in neg_reviews]

In [177]:
# Get the vectors
neg_vectors = tfidf_neg.fit_transform(neg_reviews)
pos_vectors = tfidf_pos.fit_transform(pos_reviews)

In [178]:
# change num_topics
num_topics = 6

nmf_pos = NMF(n_components=num_topics)
W_pos = nmf_pos.fit_transform(pos_vectors)
H_pos = nmf_pos.components_

nmf_neg = NMF(n_components=num_topics)
W_neg = nmf_neg.fit_transform(neg_vectors)
H_neg = nmf_neg.components_

In [179]:
no_topics = num_topics
no_top_words = 6

print('Topics for POSITIVE reviews')
print('-'*39)
display_topics(nmf_pos, tfidf_pos.get_feature_names(), no_topics, no_top_words)

print('\nTopics for NEGATIVE reviews')
print('-'*39)
display_topics(nmf_neg, tfidf_neg.get_feature_names(), no_topics, no_top_words)

Topics for POSITIVE reviews
---------------------------------------
Topic 0:   0.801*order, 0.636*come, 0.505*delicious, 0.484*chicken, 0.478*try, 0.469*dish
Topic 1:   0.937*sandwich, 0.881*cheesesteak, 0.772*cheese, 0.757*pork, 0.698*steak, 0.637*roast
Topic 2:   2.287*pizza, 0.261*crust, 0.226*slice, 0.195*pizza place, 0.169*pizza pizza, 0.158*pie
Topic 3:   1.258*place, 0.877*market, 0.732*love, 0.523*terminal, 0.399*love place, 0.399*food
Topic 4:   1.551*great, 1.176*food, 0.869*service, 0.529*recommend, 0.491*amazing, 0.460*staff
Topic 5:   0.746*hour, 0.739*happy hour, 0.717*happy, 0.686*beer, 0.571*bar, 0.511*drink

Topics for NEGATIVE reviews
---------------------------------------
Topic 0:   1.161*table, 0.773*wait, 0.729*come, 0.693*minute, 0.626*ask, 0.601*reservation
Topic 1:   0.840*chicken, 0.597*taste, 0.536*salad, 0.436*sauce, 0.393*dish, 0.372*rice
Topic 2:   2.148*pizza, 0.300*slice, 0.207*pizza place, 0.205*order pizza, 0.194*crust, 0.142*cheese
Topic 3:   0.799*ba

C:\Users\AA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topics in Positive reviews Similarity:
- Topics(1) : Order , Delicious , Dish  
- Topics(2) : Sandwich , Cheese , Steak , Pork
- Topics(3) : Pizza , Food
- Topics(4) : Place , Market
- Topics(5) : Service , Staff
- Topics(6) ：Happy hour , Beer

Topics in Negative reviews Similarity:
- Topics(1) : Table , Wait
- Topics(2) : Dish , Salad , Taste
- Topics(3) : Pizza , Food
- Topics(4) : Bar , Drink
- Topics(5) : Order , Time , Delivery
- Topics(6) ：Cheesesteak, Sandwich

## Summarize NMF results and create Topics

In [180]:
pos_topics = {0:'order/delicious/dish', 
              1:'sandwich/cheese/steak/pork', 
              2:'pizza/food', 
              3:'place/market', 
              4:'service/staff', 
              5:'happy_hour/beer'}

neg_topics = {0:'table/wait', 
              1:'dish/salad/taste', 
              2:'pizza/food', 
              3:'bar/drink', 
              4:'order/time/delivery', 
              5:'cheesesteak/sandwich'}

---

## Adding Topic Weights to Pandas DataFrame

### sample weight for a review

In [181]:
print('Topic weights : ', W_pos[1])
print('Sum of weights: ', W_pos[1].sum())

Topic weights :  [0.05695678 0.01332664 0.         0.         0.02137134 0.        ]
Sum of weights:  0.09165475882145842


### Normalize the sum of weights

In [182]:
print('Normalized Topic weights : ', normalize(W_pos[0].reshape(1,-1), norm='l1'))
print('Sum of normalized weights: ', normalize(W_pos[0].reshape(1,-1), norm='l1').sum())

Normalized Topic weights :  [[0.41207147 0.33266376 0.         0.         0.03088991 0.22437486]]
Sum of normalized weights:  1.0


In [183]:
# add normalized topic weights to each review
pos_reviews_df = pd.concat([pos_reviews_df, pd.DataFrame(normalize(W_pos, norm='l1'))], axis=1)
neg_reviews_df = pd.concat([neg_reviews_df, pd.DataFrame(normalize(W_pos, norm='l1'))], axis=1)

In [184]:
pos_reviews_df = pos_reviews_df.rename(columns={0: "topic_1", 1: "topic_2" , 2: "topic_3", 3: "topic_4" , 4: "topic_5", 5: "topic_6"})
neg_reviews_df = neg_reviews_df.rename(columns={0: "topic_1", 1: "topic_2" , 2: "topic_3", 3: "topic_4" , 4: "topic_5", 5: "topic_6"})

In [185]:
# drop unecessary columns
del pos_reviews_df['review_id']
del pos_reviews_df['text']
del pos_reviews_df['target']
del pos_reviews_df['words']

In [186]:
# peek into a dataframe to ensure it worked right
pos_reviews_df.head(5)

,business_id,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,4R2KR_-FybS7oegGrXjHVg,0.412071,0.332664,0.000000,0.000000,0.030890,0.224375
1,mUIBtlWNPD7sz3rGGWQ1RA,0.621427,0.145400,0.000000,0.000000,0.233172,0.000000
2,-mIlmp5l4hKlp1tvHRdvTg,0.000000,0.700482,0.000000,0.000000,0.291873,0.007645
3,gvD09Ev1aOmphtlq07zYEA,0.494617,0.004725,0.065640,0.435018,0.000000,0.000000
4,Ou-_OQUNvBcaAoRU0XPtaQ,0.511076,0.095343,0.001416,0.000000,0.392165,0.000000


## group the reviews by businesses and calculate the average topic weights by businesses

In [187]:
# group by business_id and apply mean/count functions to columns
pos_reviews_df = pos_reviews_df.groupby('business_id').agg({'topic_1':'mean', 'topic_2':'mean', 
                                                            'topic_3':'mean', 'topic_4':'mean',
                                                            'topic_5':'mean', 'topic_6':'mean'}).reset_index()
neg_reviews_df = neg_reviews_df.groupby('business_id').agg({'topic_1':'mean', 'topic_2':'mean', 
                                                            'topic_3':'mean', 'topic_4':'mean',
                                                            'topic_5':'mean', 'topic_6':'mean'}).reset_index()

In [188]:
# read the restaurant info file
file = "Filtered_Restaurant_Dataset.csv"
restaurant_df = pd.read_csv(file)
restaurant_df.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,general_category
0,hSbwd-VP4THYYvSKQQr6Ow,George's Famous Roast Pork and Beef,1007 S 9th St,Philadelphia,PA,19147.0,39.937345,-75.158118,4.0,27,0,"{'RestaurantsReservations': 'False', 'Restaura...","Restaurants, Delis","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'...",Restaurants


### add business name & city back to the review df

In [189]:
# merge business name/city with topic data
pos_reviews_df = pos_reviews_df.merge(restaurant_df[['business_id', 'name', 'city', 'stars']], left_on='business_id', 
                               right_on='business_id')

neg_reviews_df = neg_reviews_df.merge(restaurant_df[['business_id', 'name', 'city', 'stars']], left_on='business_id', 
                               right_on='business_id')


In [192]:
# rearrange dataframe
pos_reviews_df = pos_reviews_df[['business_id' , 'name' , 'city' , 'stars' , 'topic_1' , 'topic_2' , 'topic_3' , 'topic_4' , 'topic_5' , 'topic_6']]
neg_reviews_df = neg_reviews_df[['business_id' , 'name' , 'city' , 'stars' , 'topic_1' , 'topic_2' , 'topic_3' , 'topic_4' , 'topic_5' , 'topic_6']]
pos_reviews_df.head(1)

,business_id,name,city,stars,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,-2-ih3mE8KPyeKVIzpBfPQ,SkyGarten,Philadelphia,3.5,0.168629,0.0,0.0,0.013712,0.1451,0.672559


### With the new pos_reviews_df , we can check the restaurant that is in this dataframe that how close is the restaurant to all topics.

In [194]:
topics_weights = pos_reviews_df[pos_reviews_df.business_id == '-2-ih3mE8KPyeKVIzpBfPQ'].iloc[:,-6:].values
print(restaurant_df[restaurant_df.business_id == '-2-ih3mE8KPyeKVIzpBfPQ']['name'].values[0] + ' - Positive Reviews')
print('-'*36)
for n in pos_topics:
    print('Topic {}: {:18} > {:.4f}'.format(n, pos_topics[n], topics_weights[0][n]))

SkyGarten - Positive Reviews
------------------------------------
Topic 0: order/delicious/dish > 0.1686
Topic 1: sandwich/cheese/steak/pork > 0.0000
Topic 2: pizza/food         > 0.0000
Topic 3: place/market       > 0.0137
Topic 4: service/staff      > 0.1451
Topic 5: happy_hour/beer    > 0.6726


---

### Export to CSV

In [195]:
pos_reviews_df.to_csv('Positive_Restaurant_Topics.csv')
neg_reviews_df.to_csv('Negative_Restaurant_Topics.csv')